# Interactive SMA Crossover Backtester

This notebook provides an interactive interface for exploring SMA crossover strategies with real-time parameter tuning and visualization updates.

## Cell 1: Imports and Setup

In [1]:
# In‑notebook magic
%matplotlib inline

import sys, os
sys.path.append(os.path.abspath('..'))

from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import widgets, VBox, HBox, Output, interactive_output, interact
from IPython.display import display, clear_output

from src.data_loader   import fetch_data
from src.indicators    import compute_sma
from src.signals       import generate_signals
from src.backtester    import backtest_signals
from src.metrics       import compute_cumulative_return, compute_sharpe_ratio, compute_max_drawdown

## Cell 2: Preload and Cache Data

In [2]:
# Fetch once for default; keeps data in memory for fast reuse
_default_df = fetch_data(
    'SPY',
    '2015-01-01',
    datetime.today().strftime('%Y-%m-%d')
)

def get_cached_df(ticker, start_str, end_str):
    """Return a copy of the preloaded SPY frame or fetch new data if different."""
    if ticker.upper()=='SPY' and start_str=='2015-01-01':
        return _default_df.copy()
    else:
        return fetch_data(ticker, start_str, end_str)

Fetched data for SPY from Stooq.


## Cell 3: Widgets and Layout

In [3]:
# Parameter widgets
ticker_widget           = widgets.Text(value='SPY', description='Ticker:', style={'description_width':'80px'})
start_date_widget       = widgets.DatePicker(value=datetime(2015,1,1), description='Start:')
end_date_widget         = widgets.DatePicker(value=datetime.today(),   description='End:')
fast_sma_widget         = widgets.IntSlider( value=20, min=5,   max=100,  step=5,   description='Fast SMA:')
slow_sma_widget         = widgets.IntSlider( value=50, min=10,  max=200,  step=5,   description='Slow SMA:')
initial_cash_widget     = widgets.IntText(   value=100000,        description='Cash:',  step=10000)
transaction_cost_widget = widgets.FloatSlider(value=0.001, min=0, max=0.01, step=0.0001, description='Tx Cost:')

# Output areas
metrics_output = Output()
plots_output   = Output()

# Layout
controls = VBox([
    HBox([ticker_widget, start_date_widget, end_date_widget]),
    HBox([fast_sma_widget, slow_sma_widget]),
    HBox([initial_cash_widget, transaction_cost_widget])
])

display(controls, metrics_output, plots_output)

Output()

Output()

## Cell 4: Backtest function + interactive_output

In [4]:
def run_interactive_backtest(ticker, start_date, end_date, fast_sma, slow_sma, initial_cash, transaction_cost):
    # [Function implementation from checkpoint]
    """
    Run backtest with given parameters and display results interactively.
    """
    # Validate parameters
    if fast_sma >= slow_sma:
        with metrics_output:
            clear_output()
            print("❌ Error: Fast SMA must be less than Slow SMA")
        return
    try:
        # Fetch data
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')
        df = get_cached_df(ticker, start_str, end_str)
        # Compute SMAs and signals
        df['SMA_fast'] = compute_sma(df, fast_sma)
        df['SMA_slow'] = compute_sma(df, slow_sma)
        df = generate_signals(df)
        # Run backtest
        results = backtest_signals(df, initial_cash=initial_cash, transaction_cost=transaction_cost)
        # Calculate metrics
        cum_ret = compute_cumulative_return(results['portfolio_value'])
        sharpe = compute_sharpe_ratio(results['portfolio_value'])
        max_dd = compute_max_drawdown(results['portfolio_value'])
        # Display metrics
        with metrics_output:
            clear_output()
            print(f"📊 Results for {ticker} ({start_str} to {end_str})")
            print(f"SMA Pair: {fast_sma}/{slow_sma}")
            print("=" * 50)
            print(f"Cumulative Return: {cum_ret:.2%}")
            print(f"Sharpe Ratio:      {sharpe:.2f}")
            print(f"Max Drawdown:      {max_dd:.2%}")
            print(f"Final Portfolio:   ${results['portfolio_value'].iloc[-1]:,.0f}")
        # Display plots
        with plots_output:
            clear_output()
            # Create subplots
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
            # Price and signals
            ax1.plot(df.index, df['Close'], label='Close Price', alpha=0.7)
            ax1.plot(df.index, df['SMA_fast'], label=f'SMA {fast_sma}', alpha=0.7)
            ax1.plot(df.index, df['SMA_slow'], label=f'SMA {slow_sma}', alpha=0.7)
            # Mark signals
            buy_signals = (df['signal'].diff() > 0)
            sell_signals = (df['signal'].diff() < 0)
            ax1.scatter(df.index[buy_signals], df['Close'][buy_signals], marker='^', s=50, color='green', zorder=5, label='Buy')
            ax1.scatter(df.index[sell_signals], df['Close'][sell_signals], marker='v', s=50, color='red', zorder=5, label='Sell')
            ax1.set_title('Price and Signals')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            # Equity curve
            ax2.plot(results['portfolio_value'], linewidth=2)
            ax2.set_title('Portfolio Equity Curve')
            ax2.grid(True, alpha=0.3)
            # Drawdown
            cumulative_max = results['portfolio_value'].cummax()
            drawdown = (results['portfolio_value'] - cumulative_max) / cumulative_max
            ax3.fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red')
            ax3.plot(drawdown.index, drawdown.values, linewidth=1, color='red')
            ax3.set_title('Drawdown')
            ax3.grid(True, alpha=0.3)
            # Rolling Sharpe (if enough data)
            if len(results['portfolio_value']) > 252:
                returns = results['portfolio_value'].pct_change().dropna()
                rolling_sharpe = returns.rolling(252).mean() / returns.rolling(252).std() * np.sqrt(252)
                ax4.plot(rolling_sharpe.index, rolling_sharpe.values)
                ax4.axhline(y=0, linestyle='--', alpha=0.5)
                ax4.set_title('Rolling Sharpe Ratio (252-day)')
            else:
                ax4.text(0.5, 0.5, 'Insufficient data for rolling metrics', ha='center', va='center', transform=ax4.transAxes)
                ax4.set_title('Rolling Metrics')
            ax4.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
    except Exception as e:
        with metrics_output:
            clear_output()
            print(f"❌ Error: {str(e)}")
        with plots_output:
            clear_output()
            print(f"No plots to display due to error.")

# Create interactive function
interactive_backtest = interact(
    run_interactive_backtest,
    ticker=ticker_widget,
    start_date=start_date_widget,
    end_date=end_date_widget,
    fast_sma=fast_sma_widget,
    slow_sma=slow_sma_widget,
    initial_cash=initial_cash_widget,
    transaction_cost=transaction_cost_widget
)

print("🎯 Interactive backtester ready! Adjust the parameters above to see real-time results.")

interactive(children=(Text(value='SPY', description='Ticker:', style=TextStyle(description_width='80px')), Dat…

🎯 Interactive backtester ready! Adjust the parameters above to see real-time results.


## Cell 5: Quick Parameter Presets

In [5]:
# Preset buttons
for name, (f, s, tkr) in {
    '10/30 SPY': (10,30,'SPY'),
    '20/50 QQQ': (20,50,'QQQ'),
    '50/200 SPY':(50,200,'SPY')
}.items():
    btn = widgets.Button(description=name)
    btn.on_click(lambda _, f=f, s=s, tkr=tkr: (
        setattr(fast_sma_widget, 'value', f),
        setattr(slow_sma_widget, 'value', s),
        setattr(ticker_widget,    'value', tkr)
    ))
    display(btn)

Button(description='10/30\xa0SPY', style=ButtonStyle())

Button(description='20/50\xa0QQQ', style=ButtonStyle())

Button(description='50/200\xa0SPY', style=ButtonStyle())

## Cell 6: Usage Instructions
- Adjust the controls above to re‑run the backtest instantly.  
- Preset buttons will jump to common SMA pairs.  
- Invalid inputs show errors in the metrics pane.